In [1]:
## package setup and import
import numpy as np
import datajoint as dj
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from pathlib import Path
from scipy.io import loadmat

In [4]:
## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [5]:
datastorefolder = 'D:\Dropbox\BpodInfoseek\Data\''

dj.config['stores'] = {
    'raw_data':{
        'protocol':'file',
        'location': os.path.join(datastorefolder,'raw_data')
    }
}

dj.config.save_global()

In [6]:
schema = dj.schema('behavioral')
schema.drop(True)
schema = dj.schema('behavioral')

In [7]:
@schema
class Mice(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : varchar(100) 
    ---
    sex = 'U': enum('F','M','U')
    
    geno = 'C57BL6' : varchar(50)
    """

#     DOB = null : date  # YYYY-MM-DD
#     imaging = null : int

# format for entry of new mouse with new attributes, load later on will create new mouse if not previously entered but will be missing experimental params
mice = Mice()

mice.insert([{'mouse_id' : 'JB366', 'sex' : 'M'},{'mouse_id' : 'JB367','sex' :'F'},{'mouse_id' : 'JB368', 'sex' : 'M'},{'mouse_id' : 'JB369', 'sex' : 'M'}],skip_duplicates=True)
mice

# dj.ERD(schema)

mouse_id,sex,geno
JB366,M,C57BL6
JB367,F,C57BL6
JB368,M,C57BL6
JB369,M,C57BL6


In [8]:
# BehaviorSession.drop(True)
@schema
class BehaviorSession(dj.Manual):
    definition = """
    # Infoseek behavior .mat files from a folder
    filename : varchar(127)
    -> Mice
    date : date
    time : time
    protocol : varchar(127)
    ---
    filepath : varchar(127)
    """

In [17]:
# List all data files in directory using pathlib for insertion in manual table
files = []
# datafolder = Path('D:\Dropbox\BpodInfoseek\Data\PythonTest')
datafolder = Path(r'C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest')
for file in datafolder.glob('JB*.mat'):
    if file.stat().st_size>3000: # leaves out garbage files!
        thisfile = {}
        thisfile["filename"] = file.name
        thisfile["filepath"] = str(file.parent)
        mouseID,protocol,datestr,timestr = file.stem.split('_')
        thisfile["mouse_id"] = mouseID
        thisfile["protocol"] = protocol
        date_obj = datetime.datetime.strptime(datestr, '%Y%m%d')
        thisfile["date"] = date_obj.date()
        time_obj = datetime.datetime.strptime(timestr, '%H%M%S')
#         time_obj = datetime.datetime.strptime(timestr[:len(timestr)-4], '%H%M%S')
        thisfile["time"] = time_obj.time()
        files.append(thisfile)

In [18]:
behaviorSession = BehaviorSession()
behaviorSession.insert(files,skip_duplicates=True)
behaviorSession

filename,mouse_id,date,time,protocol,filepath
JB366_TestInfoseek_20200916_113754.mat,JB366,2020-09-16,11:37:54,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200916_115834.mat,JB366,2020-09-16,11:58:34,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200917_103906.mat,JB366,2020-09-17,10:39:06,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200918_094748.mat,JB366,2020-09-18,9:47:48,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200921_112705.mat,JB366,2020-09-21,11:27:05,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200922_103312.mat,JB366,2020-09-22,10:33:12,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200923_101021.mat,JB366,2020-09-23,10:10:21,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200924_100843.mat,JB366,2020-09-24,10:08:43,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200924_102455.mat,JB366,2020-09-24,10:24:55,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest
JB366_TestInfoseek_20200925_095727.mat,JB366,2020-09-25,9:57:27,TestInfoseek,C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest


In [ ]:
############################################
# this should be a query later
############################################

@schema
class DateTable(dj.Imported):
    definition = """
    -> MouseAttributes
    date : date
    ---
    """
    
    def make(self,key):
        mouse_id = (FileTable & key).fetch()['mouse_id']
        date = (FileTable & key).fetch()['date']
        for i in np.arange(len((FileTable & key).fetch()['date'])):
            key['mouse_id'] = mouse_id[i]
            key['date'] = date[i]
            self.insert1(key,skip_duplicates=True)
        
DateTable.populate()
DateTable()

In [19]:
filename = r'JB366_TestInfoseek_20200916_113754.mat'
filepath = r'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,filename)
print(newpath)

D:\Dropbox\BpodInfoseek\Data\PythonTest\JB366_TestInfoseek_20200916_113754.mat


In [23]:
# SessionData.drop(True)

# Save raw data

@schema
class NewData(dj.Imported):
    definition = """
    -> BehaviorSession
    ---
    average_frame   : longblob     # average fluorescence across frames
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        print('key is', key)

In [24]:
NewData.populate()

DataJointError: Populate cannot be called during a transaction.

In [ ]:
# create table that compiles water and number of trials per experimental session

@schema
class SessionWater(dj.Computed):
    definition = """
    -> TestingSession
    ---
    water_vol_ul : int
    drop_size : int
    num_trials : int
    """
    
    def make(self,key):
        dropVol = 4 # mL
        data = ((SessionData & key).fetch1()['raw_data'])
        trialEvents = data['RawEvents'][0][0][0][0][0]
        numDrops = []
        numTrials = len(trialEvents)
        for i in np.arange(len(trialEvents)):
            try:
                dropCount = len(trialEvents[i][0][0][1]['GlobalTimer3_End'][0][0][0])
            except ValueError:
                try:
                    dropCount = len(trialEvents[i][0][0][1]['GlobalTimer4_End'][0][0][0])
                except ValueError:
                    dropCount = 0

            numDrops.append(dropCount)

        totalDrops = sum(numDrops)
        
        key['water_vol_ul'] = totalDrops*dropVol
        key['drop_size'] = dropVol
        key['num_trials'] = numTrials

        self.insert1(key)
        
SessionWater.populate()
SessionWater()

In [ ]:
# table with day summary of total water given through experimental trials for individual mice each day

@schema
class DaySummaries(dj.Imported):
    definition = """
    -> DateTable
    ---
    total_water_ul : int
    """
    
    def make(self,key):
        mouseID = (DateTable & key).fetch()['mouse_id']
        dates = (DateTable & key).fetch()['date']
        for i in mouseID:
            for j in dates:
                dayWater = sum((SessionWater & 'mouse_id = ' + str('"' + i + '"') & 'date = ' + str('"' + str(j) + '"')).fetch()['water_vol_ul'])
                key['total_water_ul'] = dayWater

        self.insert1(key)    

DaySummaries.populate()
DaySummaries()

In [ ]:
# @schema
# class OutcomesPlot(dj.Computed):
#     definition = """
#     -> SessionData
#     ---
#     num_trials : int
#     outcomes : longblob
#     """
    
#     def make(self,key):
#         data = ((SessionData & key).fetch1()['raw_data'])
#         outcomes = pd.DataFrame(data['Outcomes'][0][0])
#         numTrials = len(outcomes)
#         key['num_trials'] = numTrials
#         key['outcomes'] = outcomes
#         self.insert1(key)

# OutcomesPlot.populate()
# OutcomesPlot()

In [ ]:
@schema 
class SessionProcessed(dj.Imported):
    definition = """
    -> SessionData
    ---
    trial_types : longblob
    outcomes : longblob
    orig_trial_types : longblob
    reward_types : longblob
    n_events : int
    event_names : longblob
    info : longblob
    n_trials : int
    raw_events : longblob
    session_states : longblob
    session_events : longblob
    raw_data : longblob
    trial_start_timestamps : longblob
    trial_end_timestamps : longblob
    settings_file : longblob
    notes : longblob
    marker_codes : longblob
    session_trial_settings : longblob
    session_all_trial_types : longblob
    """
    
    
    def make(self,key):
        data = ((SessionData & key).fetch1()['raw_data'])
        
        trial_types = pd.DataFrame(data['TrialTypes'][0][0])
        outcomes = pd.DataFrame(data['Outcomes'][0][0])
        orig_trial_types = pd.DataFrame(data['OrigTrialTypes'][0])
        reward_types = pd.DataFrame(data['RewardTypes'][0])
        n_events = (data['nEvents'][0][0][0])
        event_names = pd.DataFrame(data['EventNames'][0][0])
        info = pd.DataFrame(data['Info'][0][0])
        n_trials = (data['nTrials'][0][0][0])
        raw_events = pd.DataFrame(data['RawEvents'][0][0][0][0][0])
        raw_data = pd.DataFrame(data['RawData'][0][0])
        trial_start_timestamp = pd.DataFrame(data['TrialStartTimestamp'][0][0])
        trial_end_timestamp = pd.DataFrame(data['TrialEndTimestamp'][0][0])
        settings_file = pd.DataFrame(data['SettingsFile'][0][0])
        notes = pd.DataFrame(data['Notes'][0][0])
        marker_codes = pd.DataFrame(data['MarkerCodes'][0][0])
        session_trial_settings = pd.DataFrame(data['TrialSettings'][0][0])
        session_trial_types = pd.DataFrame(data['AllTrialTypes'][0][0])
        
        states = []
        events = []
        for i in np.arange(len((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0])):
            states.append((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0][i]['States'])
            events.append((SessionData & key).fetch1()['raw_data']['RawEvents'][0][0][0][0][0][i]['Events'])
        session_states = pd.DataFrame(states)
        session_events = pd.DataFrame(events)
        
        key['trial_types'] = trial_types
        key['outcomes'] = outcomes
        key['orig_trial_types'] = orig_trial_types
        key['reward_types'] = reward_types
        key['n_events'] = n_events
        key['event_names'] = event_names
        key['info'] = info
        key['n_trials'] = n_trials
        key['raw_events'] = raw_events
        key['session_states'] = session_states
        key['session_events'] = session_events
        key['raw_data'] = raw_data
        key['trial_start_timestamps'] = trial_start_timestamp
        key['trial_end_timestamps'] = trial_end_timestamp
        key['settings_file'] = settings_file
        key['notes'] = notes
        key['marker_codes'] = marker_codes
        key['session_trial_settings'] = session_trial_settings
        key['session_all_trial_types'] = session_trial_types
        
        self.insert1(key)

SessionProcessed.populate()
SessionProcessed()

In [ ]:
@schema
class TrialData(dj.Imported):
    definition = """
    -> SessionProcessed
    trial_num : int
    ---
    trial_type : int
    outcome : int
    trial_states : longblob
    trial_events : longblob
    trial_start_timestamp : float
    trial_end_timestamp : float
    original_state_names_by_number : longblob
    original_state_data : longblob
    original_event_data : longblob
    original_state_timestamps : longblob
    original_event_timestamps : longblob
    state_machine_error_codes : longblob
    trial_settings : longblob
    all_trial_types : longblob
    """
    
    def make(self,key):
        data = ((SessionProcessed & key))
        for i in np.arange((SessionProcessed & key).fetch1()['n_trials']):
            key['trial_num'] = i
            key['trial_type'] = ((SessionProcessed & key).fetch1()['trial_types'][0][i])
            key['outcome'] = ((SessionProcessed & key).fetch1()['outcomes'][0][i])
            key['trial_states'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_states'].iloc[i][0][0][0][0])
            key['trial_events'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_events'].iloc[i][0][0][0][0])
            key['trial_start_timestamp'] = ((SessionProcessed & key).fetch1()['trial_start_timestamps'])[0][i]
            key['trial_end_timestamp'] = ((SessionProcessed & key).fetch1()['trial_end_timestamps'])[0][i]            
            key['original_state_names_by_number'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['OriginalStateNamesByNumber'][0][0][i])
            key['original_state_data'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['OriginalStateData'][0][0][i])
            key['original_event_data'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['OriginalEventData'][0][0][i])
            key['original_state_timestamps'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['OriginalStateTimestamps'][0][0][i])
            key['original_event_timestamps'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['OriginalEventTimestamps'][0][0][i])
            key['state_machine_error_codes'] = pd.DataFrame(((SessionProcessed & key).fetch1()['raw_data'])['StateMachineErrorCodes'][0][0][i])
            key['trial_settings'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_trial_settings'].iloc[i][0][0])
            key['all_trial_types'] = pd.DataFrame((SessionProcessed & key).fetch1()['session_all_trial_types'].iloc[i][0])
            self.insert1(key)

TrialData.populate()
TrialData()

In [ ]:
@schema
class TrialStates(dj.Imported):
    definition = """
    -> TrialData
    ---
    start_trial : blob
    wait_for_center : blob
    center_delay : blob
    center_odor : blob
    center_post_odor_delay : blob
    go_cue : blob
    response : blob
    grace_period = null : blob
    wait_for_odor_l : blob
    wait_for_odor_r : blob
    odor_left : blob
    odor_right : blob
    reward_delay_left : blob
    reward_delay_right : blob
    left_port_check : blob
    left_not_present : blob
    right_port_check = null : blob
    right_not_present : blob
    left_big_reward : blob
    left_small_reward : blob
    right_big_reward : blob
    right_small_reward : blob
    outcome_delivery : blob
    incorrect_left : blob
    incorrect_right : blob
    no_choice : blob
    incorrect : blob
    timeout_odor : blob
    timeout_reward_delay : blob
    timeout_outcome : blob
    intertrial_interval : blob
    """    

    
    def make(self,key):
        trials = (TrialData & key).fetch1()['trial_states']
        start_trial = trials['StartTrial'][0][0]
        wait_for_center = trials['WaitForCenter'][0][0]
        center_delay = trials['CenterDelay'][0][0]
        center_odor = trials['CenterOdor'][0][0]
        center_post_odor_delay = trials['CenterPostOdorDelay'][0][0]
        go_cue = trials['GoCue'][0][0]
        response = trials['Response'][0][0]
        try:
            grace_period = trials['GracePeriod'][0][0]
        except:
            pass
        wait_for_odor_l = trials['WaitForOdorLeft'][0][0]
        wait_for_odor_r = trials['WaitForOdorRight'][0][0]
        odor_left = trials['OdorLeft'][0][0]
        odor_right = trials['OdorRight'][0][0]
        reward_delay_left = trials['RewardDelayLeft'][0][0]
        reward_delay_right = trials['RewardDelayRight'][0][0]
        left_port_check = trials['LeftPortCheck'][0][0]
        right_port_check = trials['RightPortCheck'][0][0]
        left_not_present = trials['LeftNotPresent'][0][0]
        right_not_present = trials['RightNotPresent'][0][0]
        left_big_reward = trials['LeftBigReward'][0][0]
        left_small_reward = trials['LeftSmallReward'][0][0]
        right_big_reward = trials['RightBigReward'][0][0]
        right_small_reward = trials['RightSmallReward'][0][0]
        outcome_delivery = trials['OutcomeDelivery'][0][0]
        incorrect_left = trials['IncorrectLeft'][0][0]
        incorrect_right = trials['IncorrectRight'][0][0]
        no_choice = trials['NoChoice'][0][0]
        incorrect = trials['Incorrect'][0][0]
        timeout_odor = trials['TimeoutOdor'][0][0]
        timeout_reward_delay = trials['TimeoutRewardDelay'][0][0]
        timeout_outcome = trials['TimeoutOutcome'][0][0]
        intertrial_interval = trials['InterTrialInterval'][0][0]
#         for i in np.arange(len(trials)):
#                 trialstate = trials[i][0][0][0][0]
#                 start_trial[i] = (trialstate['StartTrial'])
#                 wait_for_center[i] = (trialstate['WaitForCenter'])
#                 center_delay[i] = (trialstate['CenterDelay'])
#                 center_odor[i] = (trialstate['CenterOdor'])
#                 center_post_odor_delay[i] = (trialstate['CenterPostOdorDelay'])
#                 go_cue[i] = (trialstate['GoCue'])
#                 response[i] = (trialstate['Response'])
#                 wait_for_odor_l[i] = (trialstate['WaitForOdorLeft'])
#                 wait_for_odor_r[i] = (trialstate['WaitForOdorRight'])
#                 odor_left[i] = (trialstate['OdorLeft'])
#                 odor_right[i] = (trialstate['OdorRight'])
#                 reward_delay_left[i] = (trialstate['RewardDelayLeft'])
#                 reward_delay_right[i] = (trialstate['RewardDelayRight'])
#                 left_port_check[i] = (trialstate['LeftPortCheck'])
#         #             right_port_check[i] = pd.DataFrame(trialstate['RightPortCheck'])
#                 left_not_present[i] = (trialstate['LeftNotPresent'])
#                 right_not_present[i] = (trialstate['RightNotPresent'])
#                 left_big_reward[i] = (trialstate['LeftBigReward'])
#                 left_small_reward[i] = (trialstate['LeftSmallReward'])
#                 right_big_reward[i] = (trialstate['RightBigReward'])
#                 right_small_reward[i] = (trialstate['RightSmallReward'])
#                 outcome_delivery[i] = (trialstate['OutcomeDelivery'])
#                 incorrect_left[i] = (trialstate['IncorrectLeft'])
#                 incorrect_right[i] = (trialstate['IncorrectRight'])
#                 no_choice[i] = (trialstate['NoChoice'])
#                 incorrect[i] = (trialstate['Incorrect'])
#                 timeout_odor[i] = (trialstate['TimeoutOdor'])
#                 timeout_reward_delay[i] = (trialstate['TimeoutRewardDelay'])
#                 timeout_outcome[i]  = (trialstate['TimeoutOutcome'])
#                 intertrial_interval[i] = (trialstate['InterTrialInterval'])
        
            
            
            
        key['start_trial'] = pd.DataFrame(start_trial)
        key['wait_for_center'] = pd.DataFrame(wait_for_center)
        key['center_delay'] = pd.DataFrame(center_delay)
        key['center_odor'] = pd.DataFrame(center_odor)
        key['center_post_odor_delay'] = pd.DataFrame(center_post_odor_delay)
        key['go_cue'] = pd.DataFrame(go_cue)
        key['response'] = pd.DataFrame(response)
        key['grace_period'] = pd.DataFrame(grace_period)
        key['wait_for_odor_l'] = pd.DataFrame(wait_for_odor_l)
        key['wait_for_odor_r'] = pd.DataFrame(wait_for_odor_r)
        key['odor_left'] = pd.DataFrame(odor_left)
        key['odor_right'] = pd.DataFrame(odor_right)
        key['reward_delay_left'] = pd.DataFrame(reward_delay_left)
        key['reward_delay_right'] = pd.DataFrame(reward_delay_right)
        key['left_port_check'] = pd.DataFrame(left_port_check)
        key['left_not_present'] = pd.DataFrame(left_not_present)
        key['right_port_check'] = pd.DataFrame(right_port_check)
        key['right_not_present'] = pd.DataFrame(right_not_present)
        key['left_big_reward'] = pd.DataFrame(left_big_reward)
        key['left_small_reward'] = pd.DataFrame(left_small_reward)
        key['right_big_reward'] = pd.DataFrame(right_big_reward)
        key['right_small_reward'] = pd.DataFrame(right_small_reward)
        key['outcome_delivery'] = pd.DataFrame(outcome_delivery)
        key['incorrect_left'] = pd.DataFrame(incorrect_left)
        key['incorrect_right'] = pd.DataFrame(incorrect_right)
        key['no_choice'] = pd.DataFrame(no_choice)
        key['incorrect'] = pd.DataFrame(incorrect)
        key['timeout_odor'] = pd.DataFrame(timeout_odor)
        key['timeout_reward_delay'] = pd.DataFrame(timeout_reward_delay)
        key['timeout_outcome'] = pd.DataFrame(timeout_outcome)
        key['intertrial_interval'] = pd.DataFrame(intertrial_interval)
        
        self.insert1(key)
        

TrialStates.populate()
TrialStates()

In [ ]:
@schema
class TrialEvents(dj.Imported):
    definition = """
    -> TrialData
    ---
    tup = null : longblob
    port_1_in = null : longblob
    port_1_out = null : longblob
    port_2_in = null : longblob
    port_2_out = null : longblob
    port_3_in = null : longblob
    port_3_out = null : longblob
    global_timer_1_start = null : longblob
    global_timer_1_end = null : longblob
    global_timer_2_start = null : longblob
    global_timer_2_end = null : longblob
    global_timer_3_start = null : longblob
    global_timer_3_end = null : longblob
    global_timer_4_start = null : longblob
    global_timer_4_end = null : longblob
    global_counter_1_end = null : longblob
    global_counter_2_end = null : longblob
    global_counter_3_end = null : longblob
    global_counter_4_end = null : longblob
    """
    
    def make(self,key):
        try:
            tup = (TrialData & key).fetch1()['trial_events']['Tup'][0][0]
            key['tup'] = tup
        except KeyError:
            pass
        try:
            port_1_in = (TrialData & key).fetch1()['trial_events']['Port1In'][0][0]
            key['port_1_in'] = port_1_in
        except KeyError:
            pass
        try:
            port_1_out = (TrialData & key).fetch1()['trial_events']['Port1Out'][0][0]
            key['port_1_out'] = port_1_out
        except KeyError:
            pass
        try:
            port_2_in = (TrialData & key).fetch1()['trial_events']['Port2In'][0][0]
            key['port_2_in'] = port_2_in
        except KeyError:
            pass
        try:
            port_2_out = (TrialData & key).fetch1()['trial_events']['Port2Out'][0][0]
            key['port_2_out'] = port_2_out
        except KeyError:
            pass
        try:
            port_3_in = (TrialData & key).fetch1()['trial_events']['Port3In'][0][0]
            key['port_3_in'] = port_3_in
        except KeyError:
            pass
        try:
            port_3_out = (TrialData & key).fetch1()['trial_events']['Port3Out'][0][0]
            key['port_3_out'] = port_3_out
        except KeyError:
            pass
        try:
            global_timer_1_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer1_Start'][0][0]
            key['global_timer_1_start'] = global_timer_1_start
        except KeyError:
            pass
        try:
            global_timer_1_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer1_End'][0][0]
            key['global_timer_1_end'] = global_timer_1_end
        except KeyError:
            pass
        try:
            global_timer_2_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer2_Start'][0][0]
            key['global_timer_2_start'] = global_timer_2_start
        except KeyError:
            pass
        try:
            global_timer_2_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer2_End'][0][0]
            key['global_timer_2_end'] = global_timer_2_end
        except KeyError:
            pass
        try:
            global_timer_3_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer3_Start'][0][0]
            key['global_timer_3_start'] = global_timer_3_start
        except KeyError:
            pass
        try:
            global_timer_3_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer3_End'][0][0]
            key['global_timer_3_end'] = global_timer_3_end
        except KeyError:
            pass
        try:
            global_timer_4_start = (TrialData & key).fetch1()['trial_events']['GlobalTimer4_Start'][0][0]
            key['global_timer_4_start'] = global_timer_4_start
        except KeyError:
            pass
        try:
            global_timer_4_end = (TrialData & key).fetch1()['trial_events']['GlobalTimer4_End'][0][0]
            key['global_timer_4_end'] = global_timer_4_end
        except KeyError:
            pass
        try:
            global_counter_1_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter1_End'][0][0]
            key['global_counter_1_end'] = global_counter_1_end
        except KeyError:
            pass
        try:
            global_counter_2_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter2_End'][0][0]
            key['global_counter_2_end'] = global_counter_2_end
        except KeyError:
            pass
        try:
            global_counter_3_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter3_End'][0][0]
            key['global_counter_3_end'] = global_counter_3_end
        except KeyError:
            pass
        try:
            global_counter_4_end = (TrialData & key).fetch1()['trial_events']['GlobalCounter4_End'][0][0]
            key['global_counter_4_end'] = global_counter_4_end
        except KeyError:
            pass
        
        self.insert1(key)

TrialEvents.populate()
TrialEvents()


In [ ]:
@schema
class TrialProcessed(dj.Computed):
    definition = """
    -> TrialData
    -> TrialStates
    -> TrialEvents
    ---
    total_trial_time : float
    response_time : blob
    trial_water : int
    outcome : int
    correct_trial : int
    """
    
    def make(self,key):
        
        dropVol = 4 #uL
        
        ## trial time
        total_trial_time = (TrialData & key).fetch1()['trial_end_timestamp'] - (TrialData & key).fetch1()['trial_start_timestamp']
        
        ## response time
        def None_sum(*args):
            args = [a for a in args if not a is None]
            return sum(args) if args else None

        response_time = None_sum((TrialStates & key).fetch1()['response'].iloc[1] - (TrialStates & key).fetch1()['go_cue'].iloc[1]) + ((TrialStates & key).fetch1()['grace_period'].iloc[1] - (TrialStates & key).fetch1()['grace_period'].iloc[0])[0]
        
        ## trial water
        try:
            trial_water = len((TrialEvents & key).fetch1()['global_timer_3_end'])
        except TypeError:
            try:
                trial_water = len((TrialEvents & key).fetch1()['global_timer_4_end'])
            except TypeError:
                trial_water = 0

        ## trial outcome
        outcome = (TrialData & key).fetch1()['outcome']
        
        ## error rate
        if np.isnan((TrialStates & key).fetch1()['incorrect_left'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['incorrect_right'][0][0]) != True or np.isnan((TrialStates & key).fetch1()['incorrect'][0][0]) != True:
            key['incorrect_trial'] = 1
        else: 
            key['incorrect_trial'] = 0
        
        key['outcome'] = outcome
        key['total_trial_time'] = total_trial_time
        key['response_time'] = response_time
        key['trial_water'] = trial_water * dropVol
        self.insert1(key)
        
TrialProcessed.populate()
TrialProcessed()

In [ ]:
# print(sum((TrialProcessed & 'mouse_id = "JB366"' & 'date = "2020-09-22"').fetch()['trial_water']))
# plt.plot(TrialProcessed.fetch()['response_time'])


In [ ]:
# @schema
# class AllDaySummary(dj.Computed):
#     definition = """
    
#     ---
    
#     """

In [ ]:
# # PlotTable.drop(True)
# @schema
# class PlotTable(dj.Computed):
#     definition = """
#     -> ExpDates
#     ---
#     testplot : longblob
#     """
    
#     def make(self,key):
#         plt.figure()
#         plt.title(key)
#         testplot = plt.plot((SessionData & key).fetch()['final_reward_delay'])
#         plt.ylim = (0,10)
#         key['testplot'] = testplot
#         print('plotTable', key['mouse_id'], ': populated')
#         self.insert1(key)
        
# dj.ERD(schema)

In [ ]:
@schema
class PerDaySummary(dj.Imported):
    definition = """
    -> DateTable
    ---
    day_total_water : int
    outcomes : longblob
    """
    
    
    def make(self,key):
        mouseID = (DateTable & key).fetch()['mouse_id']
        dates = (DateTable & key).fetch()['date']
        outcomes = []
        for i in mouseID:
            for j in dates:
                dayWater = sum((SessionWater & 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"').fetch()['water_vol_ul'])
                outcomes.append((TrialData & 'mouse_id = ' + '"' + i + '"' & 'date = ' + '"' + str(j) + '"').fetch()['outcome'])
        key['day_total_water'] = dayWater
        key['outcomes'] = pd.DataFrame(outcomes)
        self.insert1(key)
                          
            
# PerDaySummary.populate()
# (PerDaySummary & 'mouse_id = "JB366"').fetch()['outcomes'][0]
PerDaySummary()


In [ ]:
schema.spawn_missing_classes()
